In [4]:
import pandas as pd

# A) Load CSV
df = pd.read_csv("AQI Bangladesh.csv")

# B) Parse datetime (handles "2022-08-05T00:00" and "2022-08-05T00:00:00")
df["datetime"] = pd.to_datetime(df["datetime"].astype(str).str.strip(), errors="coerce")

# Drop rows where datetime failed to parse
df = df.dropna(subset=["datetime"])

# C) Drop city + drop features
df = df[df["city_name"] != "Gaurnadi"].copy()
df = df.drop(columns=["carbon_dioxide", "aqi"], errors="ignore")

# D) Restrict time window
start = pd.Timestamp("2023-01-01 00:00:00")
end   = pd.Timestamp("2025-11-23 23:00:00")
df = df[(df["datetime"] >= start) & (df["datetime"] <= end)].copy()

# E) Save base dataset
out_path = "base_dataset_2023_2025.csv"
df.to_csv(out_path, index=False)

# F) Quick check
print("Saved:", out_path)
print("Rows:", len(df))
print("Cities:", df["city_name"].nunique())
print("Time range:", df["datetime"].min(), "→", df["datetime"].max())
print("Columns:", df.columns.tolist())

df.head()

Saved: base_dataset_2023_2025.csv
Rows: 736368
Cities: 29
Time range: 2023-01-01 00:00:00 → 2025-11-23 23:00:00
Columns: ['city_id', 'city_name', 'lat', 'lon', 'datetime', 'pm10', 'pm2_5', 'carbon_monoxide', 'nitrogen_dioxide', 'sulphur_dioxide', 'ozone']


,city_id,city_name,lat,lon,datetime,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone
3576,7701354,Azimpur,23.7298,90.3854,2023-01-01 00:00:00,220.7,154.1,1190.0,87.2,38.2,3.0
3577,7701354,Azimpur,23.7298,90.3854,2023-01-01 01:00:00,213.7,149.0,1227.0,84.9,35.5,4.0
3578,7701354,Azimpur,23.7298,90.3854,2023-01-01 02:00:00,205.4,143.4,1276.0,81.6,31.9,5.0
3579,7701354,Azimpur,23.7298,90.3854,2023-01-01 03:00:00,201.6,140.6,1259.0,73.7,29.6,18.0
3580,7701354,Azimpur,23.7298,90.3854,2023-01-01 04:00:00,193.2,134.6,1111.0,63.0,33.0,44.0


In [8]:
import numpy as np

# 1) Overall missingness
print("Missing counts per column:")
miss_counts = df.isna().sum().sort_values(ascending=False)
print(miss_counts)

print("\nMissing fraction per column:")
miss_frac = (df.isna().mean()).sort_values(ascending=False)
print(miss_frac)

# 2) Missingness by city (per column)
cols_check = ["pm10","pm2_5","carbon_monoxide","nitrogen_dioxide","sulphur_dioxide","ozone"]
cols_check = [c for c in cols_check if c in df.columns]

city_missing = df.groupby("city_name")[cols_check].apply(lambda g: g.isna().mean())
city_missing["avg_missing"] = city_missing.mean(axis=1)
city_missing = city_missing.sort_values("avg_missing", ascending=False)

print("\nTop 10 cities by average missing fraction across variables:")
display(city_missing.head(10))


Missing counts per column:
city_id             0
city_name           0
lat                 0
lon                 0
datetime            0
pm10                0
pm2_5               0
carbon_monoxide     0
nitrogen_dioxide    0
sulphur_dioxide     0
ozone               0
dtype: int64

Missing fraction per column:
city_id             0.0
city_name           0.0
lat                 0.0
lon                 0.0
datetime            0.0
pm10                0.0
pm2_5               0.0
carbon_monoxide     0.0
nitrogen_dioxide    0.0
sulphur_dioxide     0.0
ozone               0.0
dtype: float64

Top 10 cities by average missing fraction across variables:


,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,avg_missing
city_name,,,,,,,
Azimpur,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chhāgalnāiya,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Gafargaon,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Feni,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fatikchari,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Farīdpur,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dohār,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dinājpur,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dhaka,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# expected total hours in the window
start = pd.Timestamp("2023-01-01 00:00:00")
end   = pd.Timestamp("2025-11-23 23:00:00")
expected_hours = pd.date_range(start, end, freq="H")
expected_n = len(expected_hours)

counts = df.groupby("city_name")["datetime"].nunique().sort_values()
print("Expected hours per city:", expected_n)
print("\nMin/Max unique datetimes across cities:", counts.min(), counts.max())
display(counts.head(10))

# if any city has fewer than expected hours, list 
bad = counts[counts != expected_n]
print("\nCities with missing timestamps:", len(bad))
display(bad)

Expected hours per city: 25392

Min/Max unique datetimes across cities: 25392 25392


city_name
Azimpur        25392
Feni           25392
Fatikchari     25392
Farīdpur       25392
Dohār          25392
Dinājpur       25392
Dhaka          25392
Cox’s Bāzār    25392
Comilla        25392
Chittagong     25392
Name: datetime, dtype: int64


Cities with missing timestamps: 0


Series([], Name: datetime, dtype: int64)